### Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [1]:
from pprint import pprint
import requests
from bs4 import BeautifulSoup as bs
import lxml
import numpy as np
import unicodedata
import pandas as pd
from pymongo import MongoClient

In [2]:
client = MongoClient('localhost', 27017)
db = client['vacancy_db']
hh = db.head_hunter
sj = db.superjob

In [3]:
### функция исправления данных о заработной плате
def cur_correction(salary,currency):
    x=[]
    if salary:
        try:
            salary=int(salary)
        except Exception:
            if len(str(salary))>0:
                yy = list(salary)
                while type(salary) != int and len(yy)>0:
                    yyy = yy.pop(-1)
                    x.append(yyy)
                    try:
                        salary = int(''.join(yy))
                        cur_correction(salary)
                    except Exception:
                        pass
            else:
                pass
    currency = ''.join(x[::-1])+' '+currency
    return salary, currency

In [4]:
### функция разбора данных о заработной плате с портала head hunter
def salary_hh(salary):
    if salary[0:2]=='до':
        max_salary = ''.join(salary.split(' ')[:-1]).replace('до','')
        currency = salary.split(' ')[-1]
        min_salary = np.nan
        max_salary, currency = cur_correction(max_salary,currency)
    elif salary[0:2]=='от':
        min_salary = ''.join(salary.split(' ')[:-1]).replace('от','')
        currency = salary.split(' ')[-1]
        max_salary = np.nan
        min_salary, currency = cur_correction(min_salary,currency)
    elif '-' in salary:
        min_salary = ''.join(salary.split('-')[0].split(' '))
        currency = salary.split('-')[-1].split(' ')[-1]
        max_salary = ''.join(salary.split('-')[1].split(' ')[:-1])
        max_salary, currency = cur_correction(max_salary,currency)
        min_salary, currency = cur_correction(min_salary,currency)
    else:
        min_salary = np.nan
        max_salary = np.nan
        currency = ''
    return min_salary, max_salary, currency

def salary_sj(salary):
    if salary[0:3]=='до ':
        max_salary = ''.join(salary.split(' ')[:-1]).replace('до','').replace('|','')
        currency = salary.split(' ')[-1]
        min_salary = np.nan
        max_salary, currency = cur_correction(max_salary,currency)
    elif salary[0:3]=='от ':
        min_salary = ''.join(salary.split(' ')[:-1]).replace('от','')
        currency = salary.split(' ')[-1]
        max_salary = np.nan
        min_salary, currency = cur_correction(min_salary,currency)
    elif '—' in salary:
        min_salary = ''.join(salary.split('—')[0].split(' '))
        currency = salary.split('—')[-1].split(' ')[-1]
        max_salary = ''.join(salary.split('—')[1].split(' ')[:-1]).replace('|','')
        max_salary, currency = cur_correction(max_salary,currency)
        min_salary, currency = cur_correction(min_salary,currency)
    elif salary[:10] == 'По договор':
        min_salary = np.nan
        max_salary = np.nan
        currency = ''
    elif len(salary)>1:
        min_salary = ''.join(salary.split(' ')[:-1])
        max_salary = ''.join(salary.split(' ')[:-1])
        currency = salary.split(' ')[-1]
    else:
        min_salary = salary
        max_salary = salary
        currency = ''
    return min_salary, max_salary, currency

In [5]:
main_link_HH = 'https://hh.ru/search/vacancy'
main_link_SJ = 'https://russia.superjob.ru'#?keywords
header = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'}
# получение пользовательского запроса
# обработка запроса - приведение к виду, который используется на сайте
search_string = input()
search_query_HH = '+'.join(search_string.split())
search_query_SJ = '%20'.join(search_string.split())

 аналитик данных


In [6]:
import time
start = time.time()
vacancy_name_sj = []
salary_sjs = []
request_result_SJ = requests.get(main_link_SJ+'/vacancy/search'+'?keywords='+search_query_SJ, headers=header)
bs_result_SJ = bs(request_result_SJ.text,'lxml').find_all('div',{'class':'f-test-vacancy-item'})
i=2
while i!=-1:
    try:
        request_result_SJ_page = requests.get(main_link_SJ+f'/vacancy/search?page={i}&keywords='+search_query_SJ, headers=header)
        if len(bs(request_result_SJ_page.text,'lxml').find_all('div',{'class':'f-test-vacancy-item'}))>0:
            bs_result_SJ.extend(bs(request_result_SJ_page.text,'lxml').find_all('div',{'class':'f-test-vacancy-item'}))
        else:
            i=-2
        i+=1
    except:
        i=-1


vacancies_sj = [i.getText(separator='|') for i in bs_result_SJ]
vacancies_sj = [i\
                .replace('| ',' ')\
                .replace(' |',' ')\
                .replace('\xa0','')\
                .replace('|от||','||от ')\
                .replace('||₽|',' руб.||')\
                .replace('||—||','—')\
                .replace('|-|','-')\
                .replace('Опыт не нужен|','')\
                .replace('По договорённости','|По договорённости|')\
                for i in vacancies_sj]

for k,i in enumerate(vacancies_sj):
    z=0
    while z<20:
        try:
            if 'руб.' in str(i.split('|')[z]) and str(i.split('|')[z])[0] in [str(i) for i in range(10)]:
                i = i.split('|')
                i[z]='|' + i[z]
                i = '|'.join(i)
            i=i.replace('|до||','||до ')
            vacancy_name_sj.append(i.split('||')[0].replace('|',''))
            if len(i.split('||'))>1:
                salary_sjs.append(i.split('||')[1])
            else:
                salary_sjs.append('')
            vacancies_sj[k]=i
            break
        except Exception:
            pass
        z+=1

for k,i in enumerate(salary_sjs):
    salary_min, salary_max, currency = salary_sj(i)
    salary_sjs[k]=(salary_min, salary_max, currency)

urls_sj = []
for i in bs_result_SJ:
    urls_sj.append(main_link_SJ+i.find('a',{'class':'icMQ_'}).get('href'))

ids_sj = []
for i in urls_sj:
    ids_sj.append(i.split('.html')[0].split('-')[-1])
vac_sj_json = {}
i=0
while i<len(ids_sj):
    vac_sj_json[i]={'vacancy_name':vacancy_name_sj[i],'salary_min':salary_sjs[i][0],'salary_max':salary_sjs[i][1],'salary_currency':salary_sjs[i][2],\
                   'link':urls_sj[i],'IDs':ids_sj[i],'source_domain':'SuperJob','Search_query':search_string}
    i+=1
print(time.time() - start)

34.87605690956116


##### SUPER_JOB
* название вакансии - vacancy_name_sj
* заработная плата - salary_sjs
* ссылка - urls_sj
* id - ids_sj
* домен super_job

##### Head_Hunter
* название вакансии - vacancy_name_hh
* заработная плата - salary_hhs
* ссылка - urls_hh
* id - ids
* домен head_hunter

In [7]:
request_result_HH = requests.get(main_link_HH+'?text='+search_query_HH, headers=header)
bs_result_HH = bs(request_result_HH.text,'lxml').find('div',{'class':'vacancy-serp'})
i=1
while i:
  try:
    bs_result_HH_page = bs(requests.get(main_link_HH+'?text='+search_query_HH+'&page='+str(i), headers=header).text,'lxml').find('div',{'class':'vacancy-serp'})
    bs_result_HH.extend(bs_result_HH_page)
    i+=1
  except Exception:
    break
vacancy_name_hh = []
urls_hh = []
ids_hh = []
salary_hhs = []
for i in bs_result_HH.find_all('a',{'data-qa':'vacancy-serp__vacancy-title'}):
    vacancy_name_hh.append(i.getText())
for i in bs_result_HH.find_all('a',{'data-qa':'vacancy-serp__vacancy-title'}):
    urls_hh.append(i.get('href').split('?')[0])

for i in urls_hh:
    try:
        ids_hh.append(i.split('spb.hh.ru/vacancy/')[1])
    except:
        try:
            ids_hh.append(i.split('https://hhcdn.ru/click')[1])
        except:
            pass

for i in bs_result_HH.find_all('a',{'data-qa':'vacancy-serp__vacancy-title'}):
    if bs_result_HH.find('div',{'class':'vacancy-serp-item__sidebar'}):
        try:
            salary_hhs.append(unicodedata.normalize('NFKD',i.find_parents()[4].find('span',{'data-qa':'vacancy-serp__vacancy-compensation'}).getText()))
        except Exception:
            salary_hhs.append('')

for k,i in enumerate(salary_hhs):
    salary_min, salary_max, currency = salary_hh(i)
    salary_hhs[k]=(salary_min, salary_max, currency)            

vac_hh_json = {}
i=0
while i<len(ids_hh):
    vac_hh_json[i]={'vacancy_name':vacancy_name_hh[i],'salary_min':salary_hhs[i][0],'salary_max':salary_hhs[i][1],'salary_currency':salary_hhs[i][2],\
                   'link':urls_hh[i],'IDs':ids_hh[i],'source_domain':'HeadHunter','Search_query':search_string}
    i+=1

In [8]:
for i in vac_sj_json:
    sj.update_one({"vac_id":vac_sj_json[i]['IDs']}, {"$set":vac_sj_json[i]},upsert=True)
for i in vac_hh_json:
    hh.update_one({"vac_id":vac_hh_json[i]['IDs']}, {"$set":vac_hh_json[i]},upsert=True)

In [9]:
# для примера результата: обновлена / добавлена последняя новость, сохраненная в vac_hh_json
for i in hh.find({'vac_id':vac_hh_json[list(vac_hh_json)[-1]]['IDs']}):
    print(i)

{'_id': ObjectId('5e735e172bfea7e4e62efb88'), 'vac_id': '35999775', 'IDs': '35999775', 'Search_query': 'аналитик данных', 'link': 'https://spb.hh.ru/vacancy/35999775', 'salary_currency': '  руб.', 'salary_max': 50000, 'salary_min': 35000, 'source_domain': 'HeadHunter', 'vacancy_name': 'Менеджер-аналитик'}
